In [63]:
#Import packages
import json
import sklearn
import numpy as np
import pandas as pd

In [64]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.decomposition import IncrementalPCA, PCA, SparsePCA
from sklearn.feature_selection import mutual_info_classif, f_classif, SelectKBest, chi2, SelectPercentile
from sklearn.model_selection import KFold

In [65]:
from sklearn.preprocessing import PolynomialFeatures, PowerTransformer

In [66]:
#Load datasets
with open("domain1_train_data.json", "r") as f:
    dataset_1 = [ json.loads(line, parse_int = str) for line in f ]

with open("domain2_train_data.json", "r") as f:
    dataset_2 = [ json.loads(line, parse_int = str) for line in f ]

with open("test_data.json", "r") as f:
    testset = [ json.loads(line, parse_int = str) for line in f ]

n_samples_1 = len(dataset_1)
n_samples_2 = len(dataset_2)
n_tests = len(testset)

In [67]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

def feature_select( texts: list[str], *, vocabulary: dict = None, method="countvectorize", sparse=False, **kwargs ):
    """From a list of texts, output a dataframe of features with shape (n_samples, n_features).

     Args:
         texts (list[str]): list of strings, each item corresponding to a text.
         vocabulary (dict, optional): _description_. Defaults to None.
         method (str, optional): Method to select features. Defaults to "count-vectorizer".
         **kwargs: kwarg arguments to pass to Vectorizer classes of sklearn.
    Raises:
        ValueError: If passing an non-specified method of text feature extraction

     Returns:
         pd.DataFrame: dataframe of shape (n_samples, n_features)
    """
    #We want single digits to tokenized. This regex considers everything as a token except whitespace.
    kwargs['token_pattern'] = r'\S+' 
    if method == "countvectorize":
        vectorizer = CountVectorizer(vocabulary = vocabulary, **kwargs) if vocabulary else CountVectorizer(**kwargs)
    elif method == "tfidf":
        vectorizer = TfidfVectorizer(vocabulary = vocabulary, **kwargs) if vocabulary else TfidfVectorizer(**kwargs)
    else:
        raise ValueError(f"{method} is not a supported method.")
    if not sparse:
        X = vectorizer.fit_transform(texts)
        feature_names = vectorizer.get_feature_names_out()
        df = pd.DataFrame.sparse.from_spmatrix(data=X, columns = feature_names)
        return df, vectorizer
    else:
        X = vectorizer.fit_transform(texts)
        return X, vectorizer

In [68]:
# For separating the domains, create a binary classifier (call it domain classifier), 
# label 0 for domain 1, label 1 for domain 2.
datatexts_1 = []
datatexts_2 = []
for instance in dataset_1:
    datatexts_1 += [ " ".join(instance["text"]) ]
for instance in dataset_2:
    datatexts_2 += [ " ".join(instance["text"]) ]
    
test_texts = []
for instance in testset:
    test_texts += [" ".join(instance["text"])] 

In [69]:
#It is probably enough to only capture single-words (1-grams) for domain classifying.
X, vectorizer_domain = feature_select(texts = datatexts_1 + datatexts_2 + test_texts, 
                method='tfidf',
                ngram_range=(1,1),
                sparse=True,
                max_df=0.9995, #Ignore vocabulary appearing too frequently, probably words like "is", "are", "and", "this" etc.
                min_df=0.0005, #Ignore vocabulary that is too infrequent, as this may lead to low prediction accuracy.
                )
n_samples, n_attrs = X.shape
print(f"no. samples: {n_samples}, no. attributes: {n_attrs}")

no. samples: 22000, no. attributes: 13307


In [70]:
X = vectorizer_domain.transform( datatexts_1 + datatexts_2 ).toarray()

In [71]:
#Domain labels:
y = np.array( [ 0 ]*len(datatexts_1) + [ 1 ]*len(datatexts_2) )

In [72]:
my_selector = SelectKBest( chi2, k = 6000 )
X_reduced = my_selector.fit_transform( X, y )

In [73]:
classifier = GaussianNB()
scores = []
crossvalid = KFold(n_splits = 10, shuffle=True, random_state=20241)
for i, (train_index, test_index) in enumerate(crossvalid.split(X_reduced)):
    print(f"Fold {i}:")
    classifier.fit(X_reduced[train_index,:], y[train_index])
    score = classifier.score(X_reduced[test_index,:], y[test_index])
    scores += [score]
    print(f"score: {score}")

Fold 0:
score: 0.9922222222222222
Fold 1:
score: 0.9955555555555555
Fold 2:
score: 0.9972222222222222
Fold 3:
score: 0.9938888888888889
Fold 4:
score: 0.9955555555555555
Fold 5:
score: 0.9972222222222222
Fold 6:
score: 0.9966666666666667
Fold 7:
score: 0.9955555555555555
Fold 8:
score: 0.9961111111111111
Fold 9:
score: 1.0


In [74]:
#Gaussian NB does pretty well at domain classifying, so let's train on the whole dataset:
domain_classifier = GaussianNB()
domain_classifier.fit(X_reduced, y)

GaussianNB()

In [75]:
#Now that we have a domain classifyer, we can build a separate ML model for each domain:
#This time include up to n-grams.
n = 2
_, vectorizer_1 = feature_select(texts = datatexts_1 + test_texts, 
                method='tfidf',
                ngram_range=(1,n), 
                sparse=False,
                max_df=0.9975, #Ignore vocabulary appearing too frequently, probably words like "is", "are", "and", "this" etc.
                min_df=0.0025, #Ignore vocabulary that is too infrequent, as this may lead to low prediction accuracy.
                )
X_1 = vectorizer_1.transform( datatexts_1 ).toarray()
n_samples, n_attrs = X_1.shape
print(f"no. samples: {n_samples}, no. attributes: {n_attrs}")

no. samples: 5000, no. attributes: 11836


In [76]:
y_1 = np.array([ dataset_1[i]['label'] for i in range(n_samples) ])

In [78]:
scores = []
for alpha in np.arange(0.1, 1.1, 0.1):
    classifier = MultinomialNB(alpha=alpha)
    print(alpha)
    for p in np.arange(10,105,10):
        my_selector_1 = SelectPercentile( chi2, percentile = p )
        X_reduced_1 = my_selector_1.fit_transform( X_1, y_1 )
        crossvalid = KFold(n_splits = 5, shuffle=True, random_state=2024)
        for i, (train_index, test_index) in enumerate(crossvalid.split(X_reduced_1)):
            print(f"\tFold {i}:")
            classifier.fit(X_reduced_1[train_index,:], y_1[train_index])
            score = classifier.score(X_reduced_1[test_index,:], y_1[test_index])
            scores += [score]
            print(f"\tscore: {score}")
        print( f"alpha: {alpha} p: {p}, Mean: {np.mean(scores)}" )
    print()

0.1
	Fold 0:
	score: 0.809
	Fold 1:
	score: 0.811
	Fold 2:
	score: 0.803
	Fold 3:
	score: 0.841
	Fold 4:
	score: 0.819
alpha: 0.1 p: 10, Mean: 0.8166
	Fold 0:
	score: 0.831
	Fold 1:
	score: 0.818
	Fold 2:
	score: 0.828
	Fold 3:
	score: 0.849
	Fold 4:
	score: 0.835
alpha: 0.1 p: 20, Mean: 0.8244
	Fold 0:
	score: 0.824
	Fold 1:
	score: 0.822
	Fold 2:
	score: 0.835
	Fold 3:
	score: 0.851
	Fold 4:
	score: 0.835
alpha: 0.1 p: 30, Mean: 0.8273999999999998
	Fold 0:
	score: 0.815
	Fold 1:
	score: 0.814
	Fold 2:
	score: 0.82
	Fold 3:
	score: 0.855
	Fold 4:
	score: 0.839
alpha: 0.1 p: 40, Mean: 0.8276999999999999
	Fold 0:
	score: 0.812
	Fold 1:
	score: 0.815
	Fold 2:
	score: 0.809
	Fold 3:
	score: 0.846
	Fold 4:
	score: 0.832
alpha: 0.1 p: 50, Mean: 0.8267200000000001
	Fold 0:
	score: 0.796
	Fold 1:
	score: 0.806
	Fold 2:
	score: 0.793
	Fold 3:
	score: 0.844
	Fold 4:
	score: 0.819
alpha: 0.1 p: 60, Mean: 0.8242
	Fold 0:
	score: 0.784
	Fold 1:
	score: 0.79
	Fold 2:
	score: 0.78
	Fold 3:
	score: 0

KeyboardInterrupt: 

In [80]:
#Take p = 50, then train on full dataset:
classifier_1 = MultinomialNB(alpha=0.3)
p=35
my_selector_1 = SelectPercentile( chi2, percentile = p )
X_reduced_1 = my_selector_1.fit_transform( X_1, y_1 )
classifier_1.fit(X_reduced_1, y_1)

MultinomialNB(alpha=0.3)

In [86]:
#Now repeat for domain 2:
n = 2
_, vectorizer_2 = feature_select(texts = datatexts_2 + test_texts, 
                method='tfidf',
                ngram_range=(1,n), 
                sparse=False,
                max_df=0.9975, #Ignore vocabulary appearing too frequently, probably words like "is", "are", "and", "this" etc.
                min_df=0.0025, #Ignore vocabulary that is too infrequent, as this may lead to low prediction accuracy.
                )
X_2 = vectorizer_2.transform( datatexts_2 )
n_samples, n_attrs = X_2.shape
print(f"no. samples: {n_samples}, no. attributes: {n_attrs}")

no. samples: 13000, no. attributes: 14723


In [87]:
y_2 = np.array([ dataset_2[i]['label'] for i in range(n_samples) ])

In [24]:
classifier = MultinomialNB()
scores = []
for p in np.arange(10,105,10):
    my_selector_2 = SelectPercentile( chi2, percentile = p )
    crossvalid = KFold(n_splits = 5, shuffle=True, random_state=2024)
    X_reduced_2 = my_selector_2.fit_transform( X_2, y_2 ).toarray()
    for i, (train_index, test_index) in enumerate(crossvalid.split(X_reduced_2)):
        print(f"\tFold {i}:")
        classifier.fit(X_reduced_2[train_index,:], y_2[train_index])
        score = classifier.score(X_reduced_2[test_index,:], y_2[test_index])
        scores += [score]
        print(f"\tscore: {score}")
    print( f"p: {p}, Mean: {np.mean(scores)}" )
    print()

	Fold 0:
	score: 0.8830769230769231
	Fold 1:
	score: 0.8938461538461538
	Fold 2:
	score: 0.8946153846153846
	Fold 3:
	score: 0.88
	Fold 4:
	score: 0.8734615384615385
p: 10, Mean: 0.885

	Fold 0:
	score: 0.8826923076923077
	Fold 1:
	score: 0.895
	Fold 2:
	score: 0.895
	Fold 3:
	score: 0.88
	Fold 4:
	score: 0.8738461538461538
p: 20, Mean: 0.8851538461538461

	Fold 0:
	score: 0.8826923076923077
	Fold 1:
	score: 0.8946153846153846
	Fold 2:
	score: 0.8946153846153846
	Fold 3:
	score: 0.8792307692307693
	Fold 4:
	score: 0.8738461538461538
p: 30, Mean: 0.885102564102564

	Fold 0:
	score: 0.8823076923076923
	Fold 1:
	score: 0.8942307692307693
	Fold 2:
	score: 0.8946153846153846
	Fold 3:
	score: 0.8792307692307693
	Fold 4:
	score: 0.8734615384615385
p: 40, Mean: 0.8850192307692308

	Fold 0:
	score: 0.8819230769230769
	Fold 1:
	score: 0.8942307692307693
	Fold 2:
	score: 0.8946153846153846
	Fold 3:
	score: 0.8784615384615385
	Fold 4:
	score: 0.8734615384615385
p: 50, Mean: 0.8849230769230769

	Fo

In [88]:
labels_0 = []
labels_1 = []
for inst in dataset_2:
    if inst['label']=="0":
        labels_0 += [ inst ]
    else:
        labels_1 += [ inst ]

In [89]:
from random import shuffle, seed
seed(2024)

In [90]:
shuffle(labels_0)

In [91]:
dataset_undersampled = labels_0[:1500] + labels_1
len(dataset_undersampled)

3000

In [92]:
dataset_undersampled_texts = [] 
y_undersampled = []
for instance in dataset_undersampled:
    dataset_undersampled_texts += [ " ".join(instance["text"]) ]
    y_undersampled += [ int(instance['label']) ]
y_undersampled = np.array( y_undersampled )

In [93]:
X_undersampled_2 = vectorizer_2.transform( dataset_undersampled_texts )

In [96]:
scores = []
for alpha in np.arange(0.2, 1.2, 0.2):
    print(alpha)
    classifier = MultinomialNB(alpha=alpha)
    for p in np.arange(5,105,5):
        my_selector_2 = SelectPercentile( chi2, percentile = p )
        crossvalid = KFold(n_splits = 5, shuffle=True, random_state=2024)
        X_reduced_2 = my_selector_2.fit_transform( X_undersampled_2, y_undersampled ).toarray()
        for i, (train_index, test_index) in enumerate(crossvalid.split(X_reduced_2)):
            print(f"\tFold {i}:")
            classifier.fit(X_reduced_2[train_index,:], y_2[train_index])
            score = classifier.score(X_reduced_2[test_index,:], y_2[test_index])
            scores += [score]
            print(f"\tscore: {score}")
        print( f"alpha{alpha}: p: {p}, Mean: {np.mean(scores)}" )
        print()

0.2
	Fold 0:
	score: 0.85
	Fold 1:
	score: 0.83
	Fold 2:
	score: 0.8116666666666666
	Fold 3:
	score: 0.8316666666666667
	Fold 4:
	score: 0.8133333333333334
alpha0.2: p: 5, Mean: 0.8273333333333334

	Fold 0:
	score: 0.8783333333333333
	Fold 1:
	score: 0.8516666666666667
	Fold 2:
	score: 0.8316666666666667
	Fold 3:
	score: 0.8433333333333334
	Fold 4:
	score: 0.83
alpha0.2: p: 10, Mean: 0.8371666666666666

	Fold 0:
	score: 0.8833333333333333
	Fold 1:
	score: 0.8466666666666667
	Fold 2:
	score: 0.845
	Fold 3:
	score: 0.855
	Fold 4:
	score: 0.8283333333333334
alpha0.2: p: 15, Mean: 0.8420000000000001

	Fold 0:
	score: 0.89
	Fold 1:
	score: 0.8366666666666667
	Fold 2:
	score: 0.8466666666666667
	Fold 3:
	score: 0.8516666666666667
	Fold 4:
	score: 0.835
alpha0.2: p: 20, Mean: 0.8445

	Fold 0:
	score: 0.8883333333333333
	Fold 1:
	score: 0.8466666666666667
	Fold 2:
	score: 0.8466666666666667
	Fold 3:
	score: 0.84
	Fold 4:
	score: 0.8416666666666667
alpha0.2: p: 25, Mean: 0.8461333333333333

	Fo


KeyboardInterrupt



In [97]:
#Take p = 45, then train on full dataset:
classifier_2 = MultinomialNB(alpha=0.2)
p=40
my_selector_2 = SelectPercentile( chi2, percentile = p )
X_reduced_2 = my_selector_2.fit_transform( X_undersampled_2, y_undersampled ).toarray()
classifier_2.fit(X_reduced_2, y_undersampled)

MultinomialNB(alpha=0.2)

In [98]:
test_texts = []
for instance in testset:
    test_texts += [" ".join(instance["text"])] 
    
#First, domain classify
X_test = vectorizer_domain.transform( test_texts ).toarray()
X_test = my_selector.transform(X_test)
domain_predictions = domain_classifier.predict(X_test)
X_test.shape,domain_predictions.shape

((4000, 6000), (4000,))

In [99]:
#Next, classify according to domain 1 and 2 models:
test_1 = vectorizer_1.transform( test_texts ).toarray()
test_1 = my_selector_1.transform(test_1)
if_domain1_predicted = classifier_1.predict( test_1 )
if_domain1_predicted.shape

(4000,)

In [100]:
#Next, classify according to domain 1 and 2 models:
test_2 = vectorizer_2.transform( test_texts ).toarray()
test_2 = my_selector_2.transform(test_2)
if_domain2_predicted = classifier_2.predict( test_2 )
if_domain2_predicted.shape

(4000,)

In [101]:
final_predictions = []
for i in range(4000):
    if domain_predictions[i] == 0: #instance must be from domain 1 according to domain classifier
        final_predictions += [ int(if_domain1_predicted[i]) ]
    elif domain_predictions[i] == 1: #instance must be from domain 2 according to domain classifier
        final_predictions += [ int(if_domain2_predicted[i]) ]
len(final_predictions)

4000

In [102]:
predictions = pd.DataFrame( final_predictions, index=range(n_tests), columns=[ "class" ])

In [103]:
predictions.value_counts()

class
1        2229
0        1771
dtype: int64

In [104]:
predictions.to_csv("sample.csv", sep=",", header=True, index_label="id")

In [105]:
!kaggle competitions submit -c comp90051-2024s1-project-1 -f sample.csv -m "Same models as before with tuned hyperparameters."

100%|██████████████████████████████████████| 26.3k/26.3k [00:02<00:00, 12.7kB/s]
400 - Bad Request - Submission not allowed:  Your team has used its daily Submission allowance (10) today, please try again tomorrow UTC (12 hours from now).
